In [3]:
import pandas as pd  
  
# 读取Excel文件  
file_path = '专题统计_实际控制人持股变动.xlsx'  # 请替换为你的Excel文件的路径  
df = pd.read_excel(file_path, engine='openpyxl', dtype={'证券代码': str})
# 筛选实际控制人为国务院国有资产监督管理委员会的股票   
# 创建一个包含要搜索的所有关键字的列表  
keywords = ['国有', '国务院', '人民政府', '中共']  
# 使用str.contains()函数检查“实际控制人名称”列中的值是否包含关键字列表中的任何一个关键字，并使用any(axis=1)将结果合并为一行  
filtered_df = df[df['实际控制人名称'].str.contains('|'.join(keywords), na=False)]    
    
# 提取证券代码列并转换为列表    
stock_codes = filtered_df['证券代码'].astype(str).tolist()  
# 提取证券代码列并转换为列表  
stock_codes = filtered_df['证券代码'].astype(str).tolist()  
  
# 使用列表推导式筛选出以"00"开头的股票代码  
stock_codes_60_prefix = [code for code in stock_codes if code.startswith("60")]  
  
print(stock_codes_60_prefix)

['600019', '600062', '600072', '600099', '600101', '600119', '600135', '600151', '600176', '600262', '600278', '600320', '600343', '600391', '600396', '600435', '600480', '600482', '600489', '600495', '600500', '600505', '600508', '600528', '600536', '600582', '600583', '600602', '600618', '600642', '600651', '600675', '600676', '600692', '600698', '600708', '600741', '600744', '600748', '600819', '600820', '600822', '600833', '600835', '600841', '600845', '600061', '600581', '600523', '600550', '600268', '600048', '600021', '600292', '600636', '600444', '600533', '600663', '600834', '600795', '600725', '600156', '601975', '600715', '600228', '600100', '600009', '600050', '600056', '600073', '600104', '600170', '600184', '600236', '600284', '600312', '600406', '600452', '600517', '600519', '600552', '600560', '600597', '600616', '600621', '600623', '600629', '600637', '600640', '600648', '600650', '600737', '600754', '600764', '600826', '600827', '601668', '600038', '600316', '600198',

In [7]:
stock_codes = []
# 股票代码列表    
for i in stock_codes_60_prefix:
    stock_codes.append('sh.'+i)
stock_codes

['sh.600019',
 'sh.600062',
 'sh.600072',
 'sh.600099',
 'sh.600101',
 'sh.600119',
 'sh.600135',
 'sh.600151',
 'sh.600176',
 'sh.600262',
 'sh.600278',
 'sh.600320',
 'sh.600343',
 'sh.600391',
 'sh.600396',
 'sh.600435',
 'sh.600480',
 'sh.600482',
 'sh.600489',
 'sh.600495',
 'sh.600500',
 'sh.600505',
 'sh.600508',
 'sh.600528',
 'sh.600536',
 'sh.600582',
 'sh.600583',
 'sh.600602',
 'sh.600618',
 'sh.600642',
 'sh.600651',
 'sh.600675',
 'sh.600676',
 'sh.600692',
 'sh.600698',
 'sh.600708',
 'sh.600741',
 'sh.600744',
 'sh.600748',
 'sh.600819',
 'sh.600820',
 'sh.600822',
 'sh.600833',
 'sh.600835',
 'sh.600841',
 'sh.600845',
 'sh.600061',
 'sh.600581',
 'sh.600523',
 'sh.600550',
 'sh.600268',
 'sh.600048',
 'sh.600021',
 'sh.600292',
 'sh.600636',
 'sh.600444',
 'sh.600533',
 'sh.600663',
 'sh.600834',
 'sh.600795',
 'sh.600725',
 'sh.600156',
 'sh.601975',
 'sh.600715',
 'sh.600228',
 'sh.600100',
 'sh.600009',
 'sh.600050',
 'sh.600056',
 'sh.600073',
 'sh.600104',
 'sh.6

In [10]:
import baostock as bs    
import pandas as pd    
import os  
  
# 登陆系统    
lg = bs.login()    
# 显示登陆返回信息    
print('login respond error_code:' + lg.error_code)    
print('login respond  error_msg:' + lg.error_msg)    
    

  
# 指定存放Excel文件的文件夹路径  
folder_path = "国企沪市的季频盈利能力"  # 替换为你的文件夹路径  
  
# 如果文件夹不存在，创建它  
if not os.path.exists(folder_path):  
    os.makedirs(folder_path)  
  
# 遍历每个股票    
for code in stock_codes:    
    print(f"Processing {code}")  
    all_data_list = []    
    # 遍历每一年的四个季度    
    for year in range(2007, 2024):  # 2024用于包括2023年的数据    
        for quarter in range(1, 5):  # 一年有四个季度    
            print(f"Processing {year} quarter {quarter}")    
            # 季频偿债能力  
            profit_list = []    
            rs_profit = bs.query_profit_data(code=code, year=year, quarter=quarter)    
            # 检查是否有数据，如果没有则跳过这一年份和季度  
            if not rs_profit.data:  
                print(f"No data found for {year} quarter {quarter}. Skipping...")  
                continue  
            
            while (rs_profit.error_code == '0') & rs_profit.next():    
                profit_list.append(rs_profit.get_row_data())    
            # 将数据添加到列表中    
            result_cash_flow = pd.DataFrame(profit_list, columns=rs_profit.fields)    
            result_cash_flow['code'] = code    
            result_cash_flow['year'] = year    
            result_cash_flow['quarter'] = quarter    
            all_data_list.append(result_cash_flow)    
              
    # 合并该股票的所有数据    
    all_data = pd.concat(all_data_list, ignore_index=True)    
    # 将该股票的数据输出到单独的Excel文件中  
    excel_file_path = os.path.join(folder_path, f"{code}.xlsx") # 创建该股票的Excel文件路径  
    all_data.to_excel(excel_file_path, index=False) # 将数据保存到该Excel文件中  
    print(f"Data for {code} saved to {excel_file_path}\n") # 输出保存信息  
        
# 登出系统    
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
Processing sh.600019
Processing 2007 quarter 1
Processing 2007 quarter 2
Processing 2007 quarter 3
Processing 2007 quarter 4
Processing 2008 quarter 1
Processing 2008 quarter 2
Processing 2008 quarter 3
Processing 2008 quarter 4
Processing 2009 quarter 1
Processing 2009 quarter 2
Processing 2009 quarter 3
Processing 2009 quarter 4
Processing 2010 quarter 1
Processing 2010 quarter 2
Processing 2010 quarter 3
Processing 2010 quarter 4
Processing 2011 quarter 1
Processing 2011 quarter 2
Processing 2011 quarter 3
Processing 2011 quarter 4
Processing 2012 quarter 1
Processing 2012 quarter 2
Processing 2012 quarter 3
Processing 2012 quarter 4
Processing 2013 quarter 1
Processing 2013 quarter 2
Processing 2013 quarter 3
Processing 2013 quarter 4
Processing 2014 quarter 1
Processing 2014 quarter 2
Processing 2014 quarter 3
Processing 2014 quarter 4
Processing 2015 quarter 1
Processing 2015 quarter 2
Processing 2015 quar